# HOCOR2020 Session 1.4 語言處理管線

## 環境設置

In [ ]:
!pip install jieba
!pip install ckiptagger[tf,gdown]
!pip install CwnGraph DistilTag CwnSenseTagger
!mkdir /content/ckiptagger
!gdown --id 1_-y2P_BrbSltZsDtw1IosYsnOOZMWIRi -O data_1.4.zip && unzip -o data_1.4.zip
!curl -OL https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big

In [3]:
import jieba
import json
jieba.set_dictionary('dict.txt.big')
with open("data_1.4/corpus-wiki.json", "r", encoding="UTF-8") as fin:
    corpus = json.load(fin)

In [4]:
corpus[:2]

[{'preproc': ['語音學（、發音：）是研究言語聲音（即語音）的語言學分支學科。',
   '狹義的語音學對應英語中發音一詞，關切的重點在語音的物理、生物、心理等具象本質，與語音的實際用途、表達意義無關。',
   '與之相對的是音韻學（或稱音系學），研究音位或語音區別特徵在某種語言中運作的抽象規則和語音的系統。',
   '廣義的語音學是指這兩大方面研究的總合。',
   '本文介紹的是狹義的語音學。',
   '語音是語音學研究的客體，指的是人類說話時發出的具體聲音，即言語的聲音。',
   '國際語音學學會所制訂的國際音標是語言學界廣泛用來標註語音的音標方案。',
   '學科分支語音學的研究範疇包括以下三類：發音語音學：此分科研究發音器官（如脣、齒、舌、聲門等）如何彼此協調動作，以發出語音。',
   '聲學語音學：此分科研究語音的物理現象，如聲波的頻率、時長、振幅等。',
   '聽覺語音學：此分科研究語音的感知歷程。',
   '歷史現代語音學的最早發展是為了發明出一套記錄語音的記號系統。',
   '世紀後期，留聲機的出現令到聲音的研究更為方便，令到語音學快速發展。'],
  'query': '語音學',
  'rawtext': '語音學（Phonetics、發音：/fəˈnɛtɪks/）是研究言語聲音（即語音）的語言學分支學科。狹義的語音學對應英語中phonetics(發音)一詞，關切的重點在語音的物理、生物、心理等具象本質，與語音的實際用途、表達意義無關。與之相對的是音韻學（或稱音系學），研究音位或語音區別特徵在某種語言中運作的抽象規則和語音的系統。廣義的語音學是指這兩大方面研究的總合。本文介紹的是狹義的語音學。\n語音是語音學研究的客體，指的是人類說話時發出的具體聲音，即言語的聲音。\n國際語音學學會所制訂的國際音標是語言學界廣泛用來標註語音的音標方案。\n\n\n== 學科分支 ==\n語音學的研究範疇包括以下三類：\n\n發音語音學 (articulatory phonetics)：此分科研究發音器官（如脣、齒、舌、聲門等）如何彼此協調動作，以發出語音。\n聲學語音學 (acoustic phonetics)：此分科研究語音的物理現象，如聲波的頻率、時長、振幅等。\n聽覺語音學 (auditory phonetics)：此分科研

## N-gram 探索

In [5]:
sentence_list = []
for article in corpus:
    sentence_list.extend(article["preproc"])


In [6]:
len(sentence_list), sentence_list[:3]

(664,
 ['語音學（、發音：）是研究言語聲音（即語音）的語言學分支學科。',
  '狹義的語音學對應英語中發音一詞，關切的重點在語音的物理、生物、心理等具象本質，與語音的實際用途、表達意義無關。',
  '與之相對的是音韻學（或稱音系學），研究音位或語音區別特徵在某種語言中運作的抽象規則和語音的系統。'])

In [7]:
from tqdm.auto import tqdm

def get_ngrams(sentences, winsize=2):
    ngram_counter = {}    
    for sent in tqdm(sentences, desc=f"win: {winsize}"):
        for idx in range(0, len(sent)-winsize, winsize):
            ngram = sent[idx: idx+winsize]
            ngram_counter[ngram] = ngram_counter.get(ngram, 0) + 1
    ngrams = sorted(ngram_counter.items(), key=lambda x: x[1], reverse=True)
    return ngrams

In [8]:
bigrams = get_ngrams(sentence_list, winsize=2)
trigrams = get_ngrams(sentence_list, winsize=3)
fourgrams = get_ngrams(sentence_list, winsize=4)

In [9]:
n_ng = 20
list(zip(bigrams[:n_ng], trigrams[:n_ng], fourgrams[:n_ng]))

[(('語言', 78), ('白羊座', 40), ('距離地球', 11)),
 (('（）', 61), ('視星等', 27), ('每年月日', 10)),
 (('研究', 51), ('語義學', 20), ('拉丁語：', 10)),
 (('星座', 51), ('語言學', 20), ('亮於等的', 10)),
 (('恆星', 51), ('金牛座', 20), ('恆星有顆', 10)),
 (('星等', 49), ('獅子座', 19), ('，最亮星', 10)),
 (('光年', 48), ('球光年', 16), ('過上中天', 10)),
 (('白羊', 47), ('室女座', 16), ('視星等的', 9)),
 (('視星', 43), ('寶瓶座', 15), ('地球光年', 9)),
 (('一個', 41), ('星等爲', 14), ('，天文符', 9)),
 (('），', 39), ('雙子座', 14), ('座中心經', 9)),
 (('星，', 38), ('人馬座', 14), ('心理語言', 8)),
 (('子座', 37), ('流星雨', 13), ('），視星', 8)),
 (('座的', 36), ('星系，', 13), ('重要主星', 8)),
 (('語法', 34), ('（），', 12), ('的語義學', 7)),
 (('語義', 34), ('每年月', 12), ('白羊座是', 7)),
 (('星系', 34), ('距離地', 12), ('絕對星等', 7)),
 (('獅子', 34), ('天秤座', 12), ('表深空天', 7)),
 (('羊座', 33), ('雙魚座', 12), ('語言學的', 6)),
 (('地球', 33), ('的研究', 11), ('生成語法', 6))]

In [10]:
target = "球光年"
for sent in sentence_list:
    if target not in sent: continue
    idx = sent.index(target)    
    print("{:>20}\t{}\t{:<20}".format(
        sent[max(idx-10, 0):idx].rjust(20, "\u3000"), target, 
        sent[idx+len(target):idx+len(target)+10]))

　　　　　　　　　　視星等的橙巨星，距地	球光年	。                   
　　　　　　　　　　　　　　　婁宿一離地	球光年	，絕對星等，光度是太          
　　　　　　　　　　　　　　　婁宿二距地	球光年	，雙星中的主星是視星          
　　　　　　　　　　等，從星視星等，離地	球光年	。                   
　　　　　　　　　　　　　　　　主星距地	球光年	，絕對星等，光譜等級          
　　　　　　　　　　較大，相隔角秒，距地	球光年	，內部從星爲視星等。          
　　　　　　　　　　　　　　　　　如離地	球光年	的天陰四就是視星等，          
　　　　　　　　　　　視星等的天陰二距地	球光年	，光譜等級，絕對星等          
　　　　　　　　　　　　　　　　　　離地	球光年	的婁宿增九爲視星等，          
　　　　　　　　　　　視星等的胃宿二距地	球光年	，光譜等級，絕對星等          
　　　　　　　　　　星等的暗淡恆星，離地	球光年	，光譜等級，絕對星等          
　　　　　　　　　　亮，視星等達到，距地	球光年	，光譜等級，光度是太          
　　　　　　　　　　視星等的速逃星，離地	球光年	遠，光譜等級，估計是          
　　　　　　　　　　視星等範圍變化，距地	球光年	。                   
　　　　　　　　　　，最亮時視星等，距地	球光年	。                   
　　　　　　　　　　亮星，視星等，距離地	球光年	，雖位於畢星團內，但          
　　　　　　　　　　亮星，視星等，距離地	球光年	。                   
　　　　　　　　　　　　　　　　　距離地	球光年	。                   
　　　　　　　　　　　　　　　　它距離地	球光年	，是最接近太陽系的星          
　　　　　　　　　　利澤，紅矮星，距離地	球光年	，目前已發現一個行星          
　　　　　　　　　　疏散星團，即，距離地	球光年	。                   
　　　　　　　　　　的星雲狀物質，距離地	球光年	。        

In [11]:
corpus[16]

{'preproc': ['雙魚座（希臘語，天文符號：）是黃道星座之一，面積平方度，佔全天面積的，在全天個星座中，面積排行第十四。',
  '雙魚座每年月日子夜中心經過上中天。',
  '雙魚座中亮於等的恆星有顆，最亮星爲右更二（雙魚座），視星等爲。',
  '現在的春分點位於霹靂五（雙魚座）附近。',
  '特點雙魚座雖然是較大的星座，但組成星座的恆星都很暗。',
  '雙魚座最容易辨認的是兩個雙魚座小環，特別是緊貼飛馬座南面由雙魚座、、、、、等恆星組成的六邊形小環，另一個雙魚座小環位於飛馬座東面，由雙魚座、、、、、等恆星組成。',
  '這個星座有一個梅西耶天體：，位於雙魚座最亮星右更二附近。',
  '神話故事希臘神話中雙魚座代表的是阿佛洛狄忒和厄洛斯在水中的化身。',
  '阿佛洛狄忒爲了逃避大地女神蓋亞之子巨神提豐攻擊而變成魚躲在尼羅河（一說幼發拉底河）。',
  '之後她發現忘記帶上自己的兒子厄洛斯一起逃走，於是又上岸找到厄洛斯。',
  '爲防止與兒子失散，她將兩人腳綁在一起，隨後兩人化爲魚形，潛進河中。',
  '事後宙斯將阿佛洛狄忒首先化身的魚提升到空中成爲南魚座，而她和厄洛斯化身的綁在一起的兩條魚則稱爲雙魚座。',
  '恆星外屏七（雙魚座），雙星，在望遠鏡中可以看到一顆呈黃色，一顆呈藍色。',
  '主星視星等，伴星爲，複合星等。',
  '外屏三（雙魚座），雙星，兩子星一顆呈淺黃色，一顆呈淺紅色。',
  '主星視星等，伴星爲，複合星等。',
  '土公二（雙魚座，雙魚座），半規則變星，亮度介於之間，變光周期爲日。',
  '雲雨增七（雙魚座，雙魚座），不規則變星，亮度介於之間。',
  '範馬南星（），白矮星，視星等，距離爲光年，是雙魚座中離地球最近的恆星。',
  '範馬南星是人類發現的第三顆白矮星，也是第一顆不在多恆星系統中發現的白矮星。',
  '重要主星表中國星官中國古代傳統將雙魚座天區包括壁宿的霹靂、雲雨、土公，奎宿的奎、外屏和婁宿的右更等星官。'],
 'query': '雙魚座',
 'rawtext': "雙魚座（Template:希臘語 moon，天文符號：♓）是黃道星座之一，面積889.42平方度，佔全天面積的2.156%，在全天88個星座中，面積排行第十四。雙魚座每年9月27日子夜中心經過上中天。雙魚座中亮於5.5等的

In [12]:
test_text = "之後她發現忘記帶上自己的兒子厄洛斯一起逃走，於是又上岸找到厄洛斯。"

In [13]:
import jieba
jieba.lcut(test_text)
user_dict=["厄洛斯"]
for w in user_dict:
    jieba.add_word(w)
jieba.lcut(test_text)

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.526 seconds.
Prefix dict has been built successfully.


['之後',
 '她',
 '發現',
 '忘記',
 '帶上',
 '自己',
 '的',
 '兒子',
 '厄洛斯',
 '一起',
 '逃走',
 '，',
 '於是',
 '又',
 '上岸',
 '找到',
 '厄洛斯',
 '。']

## DistilTag & CwnSenseTagge

In [14]:
import DistilTag, CwnSenseTagger
DistilTag.download()
CwnSenseTagger.download()

A copy of DistilTag model already exists. Use upgrade=True to overwrite
A copy of DistilTag model already exists. Use upgrade=True to overwrite


In [15]:
# DistilTag是斷詞及詞性標記用，也可以使用CkipTagger
# https://github.com/ckiplab/ckiptagger
from DistilTag import DistilTag      

# senseTag是CwnSenseTagger的主要標記函式
from CwnSenseTagger import senseTag

In [18]:
import CwnSenseTagger

In [17]:
tagger = DistilTag()
tagged = tagger.tag(test_text)
tagged

[[('之後', 'Nd'),
  ('她', 'Nh'),
  ('發現', 'VE'),
  ('忘記', 'VK'),
  ('帶上', 'VC'),
  ('自己', 'Nh'),
  ('的', 'DE'),
  ('兒子', 'Na'),
  ('厄洛斯', 'Nb'),
  ('一起', 'D'),
  ('逃走', 'VA'),
  ('，', 'COMMACATEGORY')],
 [('於是', 'Cbb'),
  ('又', 'D'),
  ('上岸', 'VA'),
  ('找到', 'VC'),
  ('厄洛斯', 'Nb'),
  ('。', 'PERIODCATEGORY')]]

In [19]:
sense_tagged = senseTag(tagged)
sense_tagged

[[['之後', 'Nd', '03056402', '晚於現在或前述事件的時段。'],
  ['她', 'Nh', '05124901', '代指自己和對方以外的單數女性第三者，指人。'],
  ['發現', 'VE', '', ''],
  ['忘記', 'VK', '05074602', '沒有給後述對象應有的重視。'],
  ['帶上', 'VC', '', ''],
  ['自己', 'Nh', '05076901', '代指主事者。'],
  ['的', 'DE', '07023402', '表領屬關係，用於修飾語和中心語間。'],
  ['兒子', 'Na', '06662901', '父母對自己所生育的男性後代的指稱。'],
  ['厄洛斯', 'Nb', '', ''],
  ['一起', 'D', '04162502', '表不同的主體在同一空間或進行同一事件。'],
  ['逃走', 'VA', '', ''],
  ['，', 'COMMACATEGORY', '', '']],
 [['於是', 'Cbb', '04030601', '連接結果句，用來承接上半句的原因。'],
  ['又', 'D', '04000903', '表事件重複發生。'],
  ['上岸', 'VA', '', ''],
  ['找到', 'VC', '06609501', '搜尋到需要的後述對象，強調動作的結果。'],
  ['厄洛斯', 'Nb', '', ''],
  ['。', 'PERIODCATEGORY', '', '']]]

## 中文詞彙網路資料

In [ ]:
import CwnGraph
from CwnGraph import CwnBase, CwnSense
CwnGraph.download()
cwn = CwnBase()

In [21]:
cwn.find_lemma("^喝$")

[<CwnLemma: 喝_1>, <CwnLemma: 喝_2>]

In [22]:
cwn.find_senses("^喝$")

[<CwnSense[05226901](喝): 將液體或糊狀的食物經由口中進入體內並吸收養分。>,
 <CwnSense[05226902](喝): 專指喝酒。>,
 <CwnSense[05226903](喝): 經歷後述包括飲食的社交聚會。>,
 <CwnSense[06048101](喝): 以兇狠的態度大聲叫喊。>,
 <CwnSense[06048102](喝): 表不屑的語氣，程度較「哼」、「呸」為低。>]

In [23]:
CwnSense("05074602", cwn).semantic_relations

[('synonym', <CwnSense[05123202](忘): 沒有給後述對象應有的重視。>, 'forward'),
 ('synonym', <CwnSense[05123202](忘): 沒有給後述對象應有的重視。>, 'reversed')]

## CKIPTagger

### Loading CKIP Model

In [ ]:
from ckiptagger import construct_dictionary, WS, POS, NER
import os
from ckiptagger import data_utils

if not os.path.exists("./ckiptagger/data"):
    data_utils.download_data_gdown("./ckiptagger")
    # 備用：如果google drive限流，可用以下這行從中研院IIS下載
    # data_utils.download_data_url("./ckiptagger")

### CKIP CoreNLP Pipeline

In [25]:
ws = WS("./ckiptagger/data")
pos = POS("./ckiptagger/data")
ner = NER("./ckiptagger/data")

In [26]:
def ckip_tag(text):
    words = ws([text])
    pos_list = pos(words)
    
    words_list = []
    for words_x, pos_list_x in zip(words, pos_list):
        print("\u3000".join(f"{w}({p})" for w, p in zip(words_x, pos_list_x)))
        words_list.append([[w, p] for w, p in zip(words_x, pos_list_x)])
    ner_list = ner(words, pos_list)
    return (words_list, ner_list[0])

In [27]:
words, ners = ckip_tag(test_text)

之後(Nd)　她(Nh)　發現(VE)　忘記(VK)　帶上(VC)　自己(Nh)　的(DE)　兒子(Na)　厄洛斯(Nb)　一起(D)　逃走(VA)　，(COMMACATEGORY)　於是(Cbb)　又(D)　上岸(VA)　找到(VC)　厄洛斯(Nb)　。(PERIODCATEGORY)


In [28]:
words, ners

([[['之後', 'Nd'],
   ['她', 'Nh'],
   ['發現', 'VE'],
   ['忘記', 'VK'],
   ['帶上', 'VC'],
   ['自己', 'Nh'],
   ['的', 'DE'],
   ['兒子', 'Na'],
   ['厄洛斯', 'Nb'],
   ['一起', 'D'],
   ['逃走', 'VA'],
   ['，', 'COMMACATEGORY'],
   ['於是', 'Cbb'],
   ['又', 'D'],
   ['上岸', 'VA'],
   ['找到', 'VC'],
   ['厄洛斯', 'Nb'],
   ['。', 'PERIODCATEGORY']]],
 {(14, 17, 'PERSON', '厄洛斯'), (29, 32, 'PERSON', '厄洛斯')})

### 準備自動標記語料

In [29]:
from tqdm.auto import tqdm
tagger = DistilTag()
for article in tqdm(corpus):
    tagged_list = []
    for seq in article["preproc"]:
        try:
            tagged = tagger.tag(seq)
            tagged_seq = []
            for tagged_sent in tagged:
                tagged_seq.extend([
                    {"word": x[0], "pos": x[1]}
                    for x in tagged_sent])
            tagged_list.append(tagged_seq)
            
        except Exception as ex:
            print(f"{ex}: ", seq)
    # uncomment the following line to tag the full corpus        
    # article["tagged"] = tagged_list
    # comment the following line to tag the full corpus
    break

In [30]:
tagged_list

[[{'pos': 'Na', 'word': '語音學'},
  {'pos': 'PARENTHESISCATEGORY', 'word': '（'},
  {'pos': 'PAUSECATEGORY', 'word': '、'},
  {'pos': 'Na', 'word': '發音'},
  {'pos': 'COLONCATEGORY', 'word': '：'},
  {'pos': 'SHI', 'word': '是'},
  {'pos': 'VE', 'word': '研究'},
  {'pos': 'Na', 'word': '言語'},
  {'pos': 'Na', 'word': '聲音'},
  {'pos': 'PARENTHESISCATEGORY', 'word': '（'},
  {'pos': 'VG', 'word': '即'},
  {'pos': 'Na', 'word': '語音'},
  {'pos': 'PARENTHESISCATEGORY', 'word': '）'},
  {'pos': 'DE', 'word': '的'},
  {'pos': 'Na', 'word': '語言學'},
  {'pos': 'Na', 'word': '分支'},
  {'pos': 'Na', 'word': '學科'},
  {'pos': 'PERIODCATEGORY', 'word': '。'}],
 [{'pos': 'Na', 'word': '狹義'},
  {'pos': 'DE', 'word': '的'},
  {'pos': 'Na', 'word': '語音學'},
  {'pos': 'VC', 'word': '對應'},
  {'pos': 'Na', 'word': '英語'},
  {'pos': 'Ng', 'word': '中'},
  {'pos': 'Na', 'word': '發音'},
  {'pos': 'Neu', 'word': '一'},
  {'pos': 'Na', 'word': '詞'},
  {'pos': 'COMMACATEGORY', 'word': '，'},
  {'pos': 'VJ', 'word': '關切'},
  {'pos': 'DE

In [31]:
with open("data_1.4/corpus-wiki.tagged.json", "r", encoding="UTF-8") as fin:
    tagged_corpus = json.load(fin)

In [32]:
tagged_corpus[0]

{'preproc': ['語音學（、發音：）是研究言語聲音（即語音）的語言學分支學科。',
  '狹義的語音學對應英語中發音一詞，關切的重點在語音的物理、生物、心理等具象本質，與語音的實際用途、表達意義無關。',
  '與之相對的是音韻學（或稱音系學），研究音位或語音區別特徵在某種語言中運作的抽象規則和語音的系統。',
  '廣義的語音學是指這兩大方面研究的總合。',
  '本文介紹的是狹義的語音學。',
  '語音是語音學研究的客體，指的是人類說話時發出的具體聲音，即言語的聲音。',
  '國際語音學學會所制訂的國際音標是語言學界廣泛用來標註語音的音標方案。',
  '學科分支語音學的研究範疇包括以下三類：發音語音學：此分科研究發音器官（如脣、齒、舌、聲門等）如何彼此協調動作，以發出語音。',
  '聲學語音學：此分科研究語音的物理現象，如聲波的頻率、時長、振幅等。',
  '聽覺語音學：此分科研究語音的感知歷程。',
  '歷史現代語音學的最早發展是為了發明出一套記錄語音的記號系統。',
  '世紀後期，留聲機的出現令到聲音的研究更為方便，令到語音學快速發展。'],
 'query': '語音學',
 'rawtext': '語音學（Phonetics、發音：/fəˈnɛtɪks/）是研究言語聲音（即語音）的語言學分支學科。狹義的語音學對應英語中phonetics(發音)一詞，關切的重點在語音的物理、生物、心理等具象本質，與語音的實際用途、表達意義無關。與之相對的是音韻學（或稱音系學），研究音位或語音區別特徵在某種語言中運作的抽象規則和語音的系統。廣義的語音學是指這兩大方面研究的總合。本文介紹的是狹義的語音學。\n語音是語音學研究的客體，指的是人類說話時發出的具體聲音，即言語的聲音。\n國際語音學學會所制訂的國際音標是語言學界廣泛用來標註語音的音標方案。\n\n\n== 學科分支 ==\n語音學的研究範疇包括以下三類：\n\n發音語音學 (articulatory phonetics)：此分科研究發音器官（如脣、齒、舌、聲門等）如何彼此協調動作，以發出語音。\n聲學語音學 (acoustic phonetics)：此分科研究語音的物理現象，如聲波的頻率、時長、振幅等。\n聽覺語音學 (auditory phonetics)：此分科研究語音的感知歷程。\n\n\